In [74]:
%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

## Exercice 8

### Question 1

In [8]:
bn=gum.BayesNet('WaterSprinkler')
print(bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


In [9]:
l, b, g, f = [ bn.add(name, 2) for name in "lbgf" ]
bn

(gum::BayesNet<double>@0x15dfb6f2e30) BN{nodes: 4, arcs: 0, domainSize: 16, dim: 8}

In [10]:
for link in [(b,l),(b,g),(f,g)]:
    bn.addArc(*link)
print(bn)

BN{nodes: 4, arcs: 3, domainSize: 16, dim: 16}


In [11]:
bn

(gum::BayesNet<double>@0x15dfb6f2e30) BN{nodes: 4, arcs: 3, domainSize: 16, dim: 16}

In [13]:
bn.cpt(b).fillWith([.1,.9])
bn.cpt(f).fillWith([.1,.9])

(gum::Potential<double>@0x15dfb6f0a70) <f:0> :: 0.1 /<f:1> :: 0.9

In [20]:
bn.cpt(l)[:]=[[1,0], [.2,.8]]

In [22]:
bn.cpt(l)

(gum::Potential<double>@0x15dfb6f09d0) <l:0|b:0> :: 1 /<l:1|b:0> :: 0 /<l:0|b:1> :: 0.2 /<l:1|b:1> :: 0.8

In [15]:
#print(bn.cpt(g))
bn.cpt(g)[0,0,:] = [.9, .1] # f=0,b=0
bn.cpt(g)[0,1,:] = [.8, .2]
bn.cpt(g)[1,0,:] = [.8, .2]
bn.cpt(g)[1,1,:] = [.2, .8]
bn.cpt(g)

(gum::Potential<double>@0x15dfb6f09b0) <g:0|b:0|f:0> :: 0.9 /<g:1|b:0|f:0> :: 0.1 /<g:0|b:1|f:0> :: 0.8 /<g:1|b:1|f:0> :: 0.2 /<g:0|b:0|f:1> :: 0.8 /<g:1|b:0|f:1> :: 0.2 /<g:0|b:1|f:1> :: 0.2 /<g:1|b:1|f:1> :: 0.8

In [23]:
ie=gum.LazyPropagation(bn)
ie.makeInference()

In [25]:
ie.posterior(l)

(gum::Potential<double>@0x15dfb6f0cf0) <l:0> :: 0.28 /<l:1> :: 0.72

### Question 2

In [26]:
ie.posterior(f)

(gum::Potential<double>@0x15dfb6f0c70) <f:0> :: 0.1 /<f:1> :: 0.9

In [29]:
pLBGF=bn.cpt("l")*bn.cpt("b")*bn.cpt("g")*bn.cpt("f")
pLBGF

(gum::Potential<double>@0x15dfb6f07f0) <f:0|b:0|g:0|l:0> :: 0.009 /<f:1|b:0|g:0|l:0> :: 0.072 /<f:0|b:1|g:0|l:0> :: 0.0144 /<f:1|b:1|g:0|l:0> :: 0.0324 /<f:0|b:0|g:1|l:0> :: 0.001 /<f:1|b:0|g:1|l:0> :: 0.018 /<f:0|b:1|g:1|l:0> :: 0.0036 /<f:1|b:1|g:1|l:0> :: 0.1296 /<f:0|b:0|g:0|l:1> :: 0 /<f:1|b:0|g:0|l:1> :: 0 /<f:0|b:1|g:0|l:1> :: 0.0576 /<f:1|b:1|g:0|l:1> :: 0.1296 /<f:0|b:0|g:1|l:1> :: 0 /<f:1|b:0|g:1|l:1> :: 0 /<f:0|b:1|g:1|l:1> :: 0.0144 /<f:1|b:1|g:1|l:1> :: 0.5184

In [30]:
pLF=pLBGF.margSumOut(["b","g"])
pLF

(gum::Potential<double>@0x15dfb6f0bd0) <f:0|l:0> :: 0.028 /<f:1|l:0> :: 0.252 /<f:0|l:1> :: 0.072 /<f:1|l:1> :: 0.648

In [31]:
pL=pLBGF.margSumOut(["b","g","f"])
pL

(gum::Potential<double>@0x15dfb6f0a10) <l:0> :: 0.28 /<l:1> :: 0.72

In [32]:
pF=pLBGF.margSumOut(["b","g","l"])
pF

(gum::Potential<double>@0x15dfb6f0710) <f:0> :: 0.1 /<f:1> :: 0.9

In [38]:
pF*pL == pLF

True

### Question 3

In [42]:
pFgivenGL=(pLBGF.margSumOut(["b"])/pLBGF.margSumOut(["b","f"]))
pFgivenGL

(gum::Potential<double>@0x15dfb6f0690) <g:0|l:0|f:0> :: 0.183099 /<g:1|l:0|f:0> :: 0.0302234 /<g:0|l:1|f:0> :: 0.307692 /<g:1|l:1|f:0> :: 0.027027 /<g:0|l:0|f:1> :: 0.816901 /<g:1|l:0|f:1> :: 0.969777 /<g:0|l:1|f:1> :: 0.692308 /<g:1|l:1|f:1> :: 0.972973

In [43]:
pFgivenG=(pLBGF.margSumOut(["b","l"])/pLBGF.margSumOut(["b","l","f"]))
pFgivenG

(gum::Potential<double>@0x15dfb6f06f0) <g:0|f:0> :: 0.257143 /<g:1|f:0> :: 0.0277372 /<g:0|f:1> :: 0.742857 /<g:1|f:1> :: 0.972263

In [44]:
# On a p(F|L,G) != p(F|G)
# Donc p(F|L,G)*p(L|G) != p(F|G)*p(L|G)
# Donc p(F,L|G) != p(F|G)*p(L|G). Donc F et L non indépendants sachant G

## Exercice 6 

In [52]:
mc=gum.BayesNet('WaterSprinkler')
print(mc)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


In [53]:
x1, x2, x3 = [ mc.add(name, 3) for name in ["x1", "x2", "x3"]]
y1, y2, y3 = [ mc.add(name, 2) for name in ["y1", "y2", "y3"]]
mc

(gum::BayesNet<double>@0x15dfb6fc480) BN{nodes: 6, arcs: 0, domainSize: 216, dim: 15}

In [54]:
for link in [(x1,x2),(x2,x3),(x1,y1),(x2, y2),(x3, y3)]:
    mc.addArc(*link)
mc

(gum::BayesNet<double>@0x15dfb6fc480) BN{nodes: 6, arcs: 5, domainSize: 216, dim: 39}

In [63]:
mc.cpt(x1).fillWith([.6,.4,0])
mc.cpt(x2)[:] = [[.3,.5, .2],[0,.3,.7],[0,0,1]]
mc.cpt(x3)[:] = [[.3,.5, .2],[0,.3,.7],[0,0,1]]

mc.cpt(y1)[:] = [[1,0],[.5,.5],[0,1]]
mc.cpt(y2)[:] = [[1,0],[.5,.5],[0,1]]
mc.cpt(y3)[:] = [[1,0],[.5,.5],[0,1]]

In [64]:
ie2=gum.LazyPropagation(mc)
ie2.makeInference()

In [65]:
ie2.posterior(x1)

(gum::Potential<double>@0x15dfb71e3e0) <x1:0> :: 0.6 /<x1:1> :: 0.4 /<x1:2> :: 0

In [68]:
p123=mc.cpt("x1")*mc.cpt("y1")*mc.cpt("x2")*mc.cpt("y2")*mc.cpt("x3")*mc.cpt("y3")
p123

(gum::Potential<double>@0x15dfb71e400) <x3:0|y3:0|x2:0|y2:0|x1:0|y1:0> :: 0.054 /<x3:1|y3:0|x2:0|y2:0|x1:0|y1:0> :: 0.045 /<x3:2|y3:0|x2:0|y2:0|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:0|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:0|x1:0|y1:0> :: 0.045 /<x3:2|y3:1|x2:0|y2:0|x1:0|y1:0> :: 0.036 /<x3:0|y3:0|x2:1|y2:0|x1:0|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:0|x1:0|y1:0> :: 0.0225 /<x3:2|y3:0|x2:1|y2:0|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:0|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:0|x1:0|y1:0> :: 0.0225 /<x3:2|y3:1|x2:1|y2:0|x1:0|y1:0> :: 0.105 /<x3:0|y3:0|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:1|y3:0|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:2|y3:0|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:2|y3:1|x2:2|y2:0|x1:0|y1:0> :: 0 /<x3:0|y3:0|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:0|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:2|y3:0|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:2|y3:1|x2:0|y2:1|x1:0|y1:0> :: 0 /<x3:0|y3:0|x2:1|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:1|x1:0|y1:0> :: 0.0225 /<x3:2|y3:0|x2:1|y2:1|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:1|x1:0|y1:0> :: 0.0225 /<x3:2|y3:1|x2:1|y2:1|x1:0|y1:0> :: 0.105 /<x3:0|y3:0|x2:2|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:0|x2:2|y2:1|x1:0|y1:0> :: 0 /<x3:2|y3:0|x2:2|y2:1|x1:0|y1:0> :: 0 /<x3:0|y3:1|x2:2|y2:1|x1:0|y1:0> :: 0 /<x3:1|y3:1|x2:2|y2:1|x1:0|y1:0> :: 0 /<x3:2|y3:1|x2:2|y2:1|x1:0|y1:0> :: 0.12 /<x3:0|y3:0|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:2|y3:0|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:2|y3:1|x2:0|y2:0|x1:1|y1:0> :: 0 /<x3:0|y3:0|x2:1|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:0|x1:1|y1:0> :: 0.0045 /<x3:2|y3:0|x2:1|y2:0|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:0|x1:1|y1:0> :: 0.0045 /<x3:2|y3:1|x2:1|y2:0|x1:1|y1:0> :: 0.021 /<x3:0|y3:0|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:2|y3:0|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:2|y3:1|x2:2|y2:0|x1:1|y1:0> :: 0 /<x3:0|y3:0|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:2|y3:0|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:2|y3:1|x2:0|y2:1|x1:1|y1:0> :: 0 /<x3:0|y3:0|x2:1|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:1|x1:1|y1:0> :: 0.0045 /<x3:2|y3:0|x2:1|y2:1|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:1|x1:1|y1:0> :: 0.0045 /<x3:2|y3:1|x2:1|y2:1|x1:1|y1:0> :: 0.021 /<x3:0|y3:0|x2:2|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:0|x2:2|y2:1|x1:1|y1:0> :: 0 /<x3:2|y3:0|x2:2|y2:1|x1:1|y1:0> :: 0 /<x3:0|y3:1|x2:2|y2:1|x1:1|y1:0> :: 0 /<x3:1|y3:1|x2:2|y2:1|x1:1|y1:0> :: 0 /<x3:2|y3:1|x2:2|y2:1|x1:1|y1:0> :: 0.14 /<x3:0|y3:0|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:0|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:0|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:1|x2:0|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:0|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:0|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:1|x2:1|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:0|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:0|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:0|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:1|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:1|y3:1|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:2|y3:1|x2:2|y2:0|x1:2|y1:0> :: 0 /<x3:0|y3:0|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:1|y3:0|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:2|y3:0|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:0|y3:1|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:1|y3:1|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:2|y3:1|x2:0|y2:1|x1:2|y1:0> :: 0 /<x3:0|y3:0|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:1|y3:0|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:2|y3:0|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:0|y3:1|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:1|y3:1|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:2|y3:1|x2:1|y2:1|x1:2|y1:0> :: 0 /<x3:0|y

In [82]:
pY=p123.margSumOut(["x1","x2","x3"])
print("Proba d'obtenir la suite aab : {}".format(round(pY[0,0,1], 3)))
pY

Proba d'obtenir la suite aab : 0.234


(gum::Potential<double>@0x15dfb6f71d0) <y3:0|y2:0|y1:0> :: 0.126 /<y3:1|y2:0|y1:0> :: 0.234 /<y3:0|y2:1|y1:0> :: 0.027 /<y3:1|y2:1|y1:0> :: 0.413 /<y3:0|y2:0|y1:1> :: 0.0045 /<y3:1|y2:0|y1:1> :: 0.0255 /<y3:0|y2:1|y1:1> :: 0.0045 /<y3:1|y2:1|y1:1> :: 0.1655

In [80]:
ie2.setEvidence({'y1':0, 'y2':0, 'y3':1})
ie2.makeInference()
ie2.posterior('x1')*ie2.posterior('x2')*ie2.posterior('x3')

(gum::Potential<double>@0x15dfb6f7250) <x3:0|x2:0|x1:0> :: 0 /<x3:1|x2:0|x1:0> :: 0.0949021 /<x3:2|x2:0|x1:0> :: 0.21353 /<x3:0|x2:1|x1:0> :: 0 /<x3:1|x2:1|x1:0> :: 0.17926 /<x3:2|x2:1|x1:0> :: 0.403334 /<x3:0|x2:2|x1:0> :: 0 /<x3:1|x2:2|x1:0> :: 0 /<x3:2|x2:2|x1:0> :: 0 /<x3:0|x2:0|x1:1> :: 0 /<x3:1|x2:0|x1:1> :: 0.0116067 /<x3:2|x2:0|x1:1> :: 0.0261152 /<x3:0|x2:1|x1:1> :: 0 /<x3:1|x2:1|x1:1> :: 0.0219238 /<x3:2|x2:1|x1:1> :: 0.0493286 /<x3:0|x2:2|x1:1> :: 0 /<x3:1|x2:2|x1:1> :: 0 /<x3:2|x2:2|x1:1> :: 0 /<x3:0|x2:0|x1:2> :: 0 /<x3:1|x2:0|x1:2> :: 0 /<x3:2|x2:0|x1:2> :: 0 /<x3:0|x2:1|x1:2> :: 0 /<x3:1|x2:1|x1:2> :: 0 /<x3:2|x2:1|x1:2> :: 0 /<x3:0|x2:2|x1:2> :: 0 /<x3:1|x2:2|x1:2> :: 0 /<x3:2|x2:2|x1:2> :: 0

In [83]:
# Sachant qu'on a observé la suite de symboles "aab", la séquence d'états la plus probable est 123, avec un proba de 0.4033